In [2]:
import pandas as pd

net_filename = 'skill_bipartite.net'
clu_filename = 'skill_partition.clu'

# 1. CSV 파일 로드
try:
    df = pd.read_csv('C:\\socialnetwork\\SocialNetwork_7\\bipartite_skill_long.csv', encoding='utf-8-sig')
    print(f"--- 파일 로드 성공: 'bipartite_skill_long.csv'")

except FileNotFoundError:
    print(f"--- [오류] 'bipartite_skill_long.csv' 파일을 찾을 수 없습니다.")
    print(f"파일을 이 스크립트와 같은 폴더로 옮긴 후 다시 실행해 주세요.")
    
else:
    # 2. 노드 리스트 생성 (기업: Partition 1, 스킬: Partition 2)
    companies = df['기업명'].unique()
    skills = df['Skill'].unique()

    company_count = len(companies)
    skill_count = len(skills)
    total_nodes = company_count + skill_count

    print(f"기업 (그룹 1) 노드 수: {company_count}")
    print(f"스킬 (그룹 2) 노드 수: {skill_count}")
    print(f"총 노드 수: {total_nodes}")

    # 3. 노드 ID 매핑 (Pajek은 숫자 ID를 사용)
    company_map = {name: i+1 for i, name in enumerate(companies)}
    skill_map = {name: i+1 + company_count for i, name in enumerate(skills)}

    # -------------------------------------------------
    # 4. (수정) .net 파일 생성 (네트워크 파일)
    # -------------------------------------------------
    try:
        with open(net_filename, 'w', encoding='utf-8') as f:
            # *Vertices [총 노드 수] 
            # (이분 그래프 정보를 여기 적지 않고, .clu 파일로 분리)
            f.write(f"*Vertices {total_nodes}\n")
            
            # 노드 ID와 레이블(이름) 작성
            for company, cid in company_map.items():
                f.write(f'{cid} "{company}"\n')
            for skill, sid in skill_map.items():
                f.write(f'{sid} "{skill}"\n')
                
            # *Edges 섹션 작성
            f.write("*Edges\n")
            for index, row in df.iterrows():
                company_id = company_map[row['기업명']]
                skill_id = skill_map[row['Skill']]
                f.write(f"{company_id} {skill_id}\n")
        
        print(f"--- 성공: '{net_filename}' 생성 완료")

    except Exception as e:
        print(f"--- [오류] .net 파일 생성 중 오류: {e}")

    # -------------------------------------------------
    # 5. (신규) .clu 파일 생성 (파티션 파일)
    # -------------------------------------------------
    try:
        with open(clu_filename, 'w', encoding='utf-8') as f:
            # *Vertices [총 노드 수]
            f.write(f"*Vertices {total_nodes}\n")
            
            # 기업 노드(1 ~ company_count)는 1번 그룹
            for i in range(company_count):
                f.write("1\n")
                
            # 스킬 노드(company_count+1 ~ total_nodes)는 2번 그룹
            for i in range(skill_count):
                f.write("2\n")
        
        print(f"--- 성공: '{clu_filename}' 생성 완료")

    except Exception as e:
        print(f"--- [오류] .clu 파일 생성 중 오류: {e}")

    print("\n이제 Pajek에서 .net 과 .clu 파일을 차례로 로드하세요.")

--- 파일 로드 성공: 'bipartite_skill_long.csv'
기업 (그룹 1) 노드 수: 1260
스킬 (그룹 2) 노드 수: 779
총 노드 수: 2039
--- 성공: 'skill_bipartite.net' 생성 완료
--- 성공: 'skill_partition.clu' 생성 완료

이제 Pajek에서 .net 과 .clu 파일을 차례로 로드하세요.


In [ ]:
import pandas as pd

# 1. CSV 파일 로드
try:
    df = pd.read_csv('C:\\socialnetwork\\SocialNetwork_7\\bipartite_skill_long.csv', encoding='utf-8-sig')
except FileNotFoundError:
    print("오류: 'bipartite_skill_long.csv' 파일을 찾을 수 없습니다.")
    # 이 else 블록은 try가 성공해야만 실행됩니다.
else:
    # 2. 노드 리스트 생성
    companies = df['기업명'].unique()
    skills = df['Skill'].unique()

    company_count = len(companies)
    skill_count = len(skills)
    total_nodes = company_count + skill_count

    print(f"기업 (Partition 1) 노드 수: {company_count}")
    print(f"스킬 (Partition 2) 노드 수: {skill_count}")

    # 3. 노드 ID 매핑
    company_map = {name: i+1 for i, name in enumerate(companies)}
    skill_map = {name: i+1 + company_count for i, name in enumerate(skills)}

    # 4. Pajek .net 파일 생성
    output_filename = 'skill_bipartite_with_partition.net'
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            # --- *Vertices 섹션 (★가장 중요) ---
            # 형식: *Vertices [총 노드 수] [1번 파티션 노드 수]
            # 여기에 이분 그래프 정보가 포함됩니다.
            f.write(f"*Vertices {total_nodes} {company_count}\n")
            
            # 노드 작성
            for company, cid in company_map.items():
                f.write(f'{cid} "{company}"\n')
            for skill, sid in skill_map.items():
                f.write(f'{sid} "{skill}"\n')
                
            # --- *Edges 섹션 ---
            f.write("*Edges\n")
            for index, row in df.iterrows():
                company_id = company_map[row['기업명']]
                skill_id = skill_map[row['Skill']]
                f.write(f"{company_id} {skill_id}\n")

        print(f"\n성공: '{output_filename}' 파일이 생성되었습니다.")
        print("이제 Pajek에서 이 '파일 1개만' 로드하세요.")

    except Exception as e:
        print(f"\n오류: .net 파일 생성 중 문제가 발생했습니다. ({e})")

기업 (Partition 1) 노드 수: 1260
스킬 (Partition 2) 노드 수: 779

성공: 'skill_bipartite_with_partition.net' 파일이 생성되었습니다.
이제 Pajek에서 이 '파일 1개만' 로드하세요.
